In [1]:
# Import Data
import pandas as pd 
import Sastrawi 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Stemming untuk bahasa indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory #Stopword bahasa indonesia
from itertools import chain # untuk menggabungkan list
import nltk # dipakai untuk tokenizing / pemisah kalimat jadi perkata
import math
import numpy as np

# 1.1 Baca Data

In [3]:
baca = pd.read_csv('hadits.csv', sep=';') # membaca data dari file csv memakai pandas
baca

,Hadits
0,Dari Abi Abdurrahman Abdillah bin Umar bin Kha...
1,Dari Abi Hamzah Anas bin Malik ra. pelayan Ras...
2,Dari Ibni Mas?ud ra. telah berkata: Telah bers...
3,Dari Abu Musa (Abdullah) bin Qais al-asy?ary r...
4,Dari Abu Bakrah (Nufa?i) bin al Harits ats Tsa...
5,Dari Anas bin Malik r.a. berkata: Rasulullah s...
6,Dari Abu Said dan Abu Hurairah r.a. berkata: R...
7,"Dari Abu Hurairah r.a. berkata, Rasulullah ber..."
8,Dari Abu Khalid (Hakim) bin hizam r.a. berkata...
9,Dari Abu Hurairah r.a. berkata: Rasulullah saw...


In [4]:
# Mengambil teks isinya saja dari file artikel.csv (tanpa judul dan index)
# dan mengubah dari bentuk Dataframe (pandas) ke List
data = baca.values.tolist()

In [5]:
# MEMBUAT VARIABEL SEBAGAI ALAT UNTUK STOPWORD DAN STEMMING
# Buat Alat Stopword
stop = StopWordRemoverFactory()
stopword = stop.create_stop_word_remover()

# Buat Alat Stemming
stem = StemmerFactory()
stemmer = stem.create_stemmer()

In [ ]:
# membuat data menjadi 1 list agar dapat di olah pada Stopword dan Stemming
data_list = list(chain.from_iterable(data))

In [ ]:
# Membuat Huruf menjadi kecil (lowercase)
data_lower = list() #untuk menampung data yang sudah dikecilkan hurufnya
for i in range(0, len(data_list)):
    data_lower.append(data_list[i].lower())

In [ ]:
baca

In [ ]:
# Contoh Stopword
wadah = list()
index = 0
for i in range(0, len(data_lower)):
    wadah.append(stopword.remove(data_lower[i]))

# 1.2 Full PraProcessing

In [ ]:
token = list()  # untuk menampung data yang sudah dipisahkan perkata
filter_kata = list()  # untuk menampung data yang sudah di filter / stopword
kata_dasar = list()  # untuk menampung data yang sudah di stemming / kata dasar
index = 0  
for kalimat in wadah:
    filter_kata.append(stopword.remove(kalimat))  # proses Stopword
    kata_dasar.append(stemmer.stem(filter_kata[index]))  # Proses Stemming
    token.append(nltk.word_tokenize(kata_dasar[index])) # Proses pemisahan dari kalimat menjadi kata
    final_kata = token
    index+=1

In [ ]:
filter_kata

### Koding dibawah ini merupakan Fungsi untuk masing-masing Proses TF-IDF

In [ ]:
# Menghitung Term Frequence -> Berapa banyak kata tersebut ada dalam dokumen
def compute_tf(word_dict, filtered_word):
    tfDict = {}
    filtered_wordcount = len(filtered_word)
    for word, count in word_dict.items():
        if count == 0:
            tfDict[word] = count
        else:
            tfDict[word] = 1 + (math.log10(count))
#         print(word, count)
#         print(tfDict)
#         print("-"*100)
    return tfDict

# Menghitung Inverse Document Frequence -> Seberapa Umum Kata Tersebut terhadap Keseluruhan Dokumen
def compute_idf(doc_list):
    print("#"*100)
    import math
    N = len(doc_list)
#     print("Banyak Dokumen:", N)

    # Count the number of documents that contain a word w
    idDict = dict.fromkeys(doc_list[0].keys(), 0)
    for doc in doc_list:
        for word, val in doc.items():
            if val > 0:
                idDict[word] += 1
#     print("Document Frequency:", idDict)

    # Devide N by decombinator above, take the log of that
    for word, val in idDict.items():
        print(word, val)
        idDict[word] = 1 + (math.log10(N/float(val)))
    return idDict


# Menghitung TF.IDF
def compute_tfidf(tf_hadits, idfs):
    tfidf = {}
    for word, val in tf_hadits.items():
        tfidf[word] = val * idfs[word]
    return tfidf

### Koding dibawah ini untuk mengeksekusi koding diatas

In [ ]:
# Data dar hasil Preprocessing di satukan menjadi 1 List
corpus = list(chain.from_iterable(final_kata))
# print('corpus:', corpus)

In [ ]:
print(corpus)

In [ ]:
# Penggabungan semua kata menjadi satu Set dan penghilangan kata yang sama
wordSet = (set().union(corpus))

print('wordSet:',wordSet)

In [ ]:
# Membuat Kamus Kata (Dictionary Word) untuk menyimpan perhitungan
# Serta memberi nilai awal 0
wordDict = []
for i in range(0, len(data_list)):
    wordDict.append(dict.fromkeys(wordSet, 0))

In [ ]:
# Proses perhitungan banyaknya Kata pada Dokumen masing2
nomor = 0
for i in range(0, len(data_list)):
    for word in final_kata[nomor]:
        wordDict[nomor][word] += 1
    nomor += 1
    
# ILUSTRASI #
# Dokument 1 :  ['sholat', 'sarana', 'panjat', 'doa', 'cukup', 'doa', 'sholat', 'ganti']
# Dokument 2 : ['sholat', 'wajib', 'laksana', 'tepat', 'waktu', 'jamaah', 'sholat', 'wajib', 'lebih', 'utama']
# Dokument 3 : ['waktu', 'mustajab', 'doa', 'doa', 'sholat', 'sering', 'laku', 'orang']
# Dokument 4 : ['doa', 'sholat', 'wajib', 'laku', 'wajib', 'orang', 'selalu', 'doa']

# wordDict[0] untuk Dokumen 1
# {'wajib': 0, 'sering': 0, 'utama': 0, 'panjat': 1, 'jamaah': 0, 
#  'mustajab': 0, 'laku': 0, 'cukup': 1, 'doa': 2, 'sholat': 2, 
# 'waktu': 0, 'selalu': 0, 'tepat': 0, 'lebih': 0, 'ganti': 1, 
# 'sarana': 1, 'laksana': 0, 'orang': 0}

print("wordDict[0]:",wordDict[0])
print("="*125)

In [ ]:
# Menghitung TF(Term Frequence)
tf = list()
for i in range(0, len(data_lower)):
    tf.append(compute_tf(wordDict[i], final_kata[i]))

print("tf[0]", tf[0])

In [ ]:
# Menghitung IDF
idfs = (compute_idf([word for word in wordDict]))

print("idfs:",idfs)
print("="*125)

In [ ]:
# Menghitung TFIDF
tf_idf = list()
for i in range(0, len(data_lower)):
    tf_idf.append(compute_tfidf(tf[i], idfs))
    
print("tf_idf[0]",tf_idf[0])
# print("tf_idf[1]",tf_idf[1])
# print("tf_idf[2]",tf_idf[2])

# Ranking Cosine Similarity [BETA]

# Cosine Similarity

## Rumus :
### cos(A,B) =  A * B / ||A|| . ||B||


Setelah melakukan Tahap Preprocessing dan Pembobotan Kata. Hasil Pembobotan perdokumen dimasukan kedalam beberapa variabel. tf_idf[0] = query; tf_idf[1] = dokumen1; tf_idf[2] = dokumen2; tf_idf[3] = dokumen3;
Skenario sebagai berikut: Kita anggap bahwa hasil tf_idf[0] adalah query yang akan di samakan dengan dokumen lainnya (dokumen 1,2,3) kita simpan tf_idf[1] sebagai dokumen1, tf_idf[2] sebagai dokumen2, tf_idf[3] sebagai dokumen3 Nantinya akan dihitung kesamaan antara query dan semua dokumen. 
kita ibaratkan: query = A dokumen1 = B dokumen2 = c dokumen3 = D
Sehingga kita akan menghitung kesamaan cos(A, B) -> kesamaan query dengan dokumen1 cos(A, C) -> kesamaan query dengan dokumen2 cos(A, D) -> kesamaan query dengan dokumen3

In [ ]:
dokumen1 = tf_idf[1] 
dokumen2 = tf_idf[2] 
dokumen3 = tf_idf[3]
dokumen4 = tf_idf[4] 
dokumen5 = tf_idf[5] 
dokumen6 = tf_idf[6]
dokumen7 = tf_idf[7] 
dokumen8 = tf_idf[8] 
dokumen9 = tf_idf[9]
dokumen10 = tf_idf[10]
dokumen11 = tf_idf[11] 
dokumen12 = tf_idf[12] 
dokumen13 = tf_idf[13]
dokumen14 = tf_idf[14] 
dokumen15 = tf_idf[15] 
dokumen16 = tf_idf[16]
dokumen17 = tf_idf[17] 
dokumen18 = tf_idf[18] 
dokumen19 = tf_idf[19]
dokumen20 = tf_idf[20]
dokumen21 = tf_idf[21] 
dokumen22 = tf_idf[22] 
dokumen23 = tf_idf[23]
dokumen24 = tf_idf[24] 
dokumen25 = tf_idf[25] 
dokumen26 = tf_idf[26]
dokumen27 = tf_idf[27] 
dokumen28 = tf_idf[28] 
dokumen29 = tf_idf[29]

In [ ]:
intersection1 = set(query.keys()) & set(dokumen1.keys())
intersection2 = set(query.keys()) & set(dokumen2.keys())
intersection3 = set(query.keys()) & set(dokumen3.keys())
intersection4 = set(query.keys()) & set(dokumen4.keys())
intersection5 = set(query.keys()) & set(dokumen5.keys())
intersection6 = set(query.keys()) & set(dokumen6.keys())
intersection7 = set(query.keys()) & set(dokumen7.keys())
intersection8 = set(query.keys()) & set(dokumen8.keys())
intersection9 = set(query.keys()) & set(dokumen9.keys())
intersection10 = set(query.keys()) & set(dokumen10.keys())
intersection11 = set(query.keys()) & set(dokumen11.keys())
intersection12 = set(query.keys()) & set(dokumen12.keys())
intersection13 = set(query.keys()) & set(dokumen13.keys())
intersection14 = set(query.keys()) & set(dokumen14.keys())
intersection15 = set(query.keys()) & set(dokumen15.keys())
intersection16 = set(query.keys()) & set(dokumen16.keys())
intersection17 = set(query.keys()) & set(dokumen17.keys())
intersection18 = set(query.keys()) & set(dokumen18.keys())
intersection19 = set(query.keys()) & set(dokumen19.keys())
intersection20 = set(query.keys()) & set(dokumen20.keys())
intersection21 = set(query.keys()) & set(dokumen21.keys())
intersection22 = set(query.keys()) & set(dokumen22.keys())
intersection23 = set(query.keys()) & set(dokumen23.keys())
intersection24 = set(query.keys()) & set(dokumen24.keys())
intersection25 = set(query.keys()) & set(dokumen25.keys())
intersection26 = set(query.keys()) & set(dokumen26.keys())
intersection27 = set(query.keys()) & set(dokumen27.keys())
intersection28 = set(query.keys()) & set(dokumen28.keys())
intersection29 = set(query.keys()) & set(dokumen29.keys())

In [ ]:
numerator1 = sum([query[x] * dokumen1[x] for x in intersection1])  # A dot B
numerator2 = sum([query[x] * dokumen2[x] for x in intersection2])  # A dot C
numerator3 = sum([query[x] * dokumen3[x] for x in intersection2])  # A dot D
numerator4 = sum([query[x] * dokumen1[x] for x in intersection1])  # A dot B
numerator5 = sum([query[x] * dokumen2[x] for x in intersection2])  # A dot C
numerator6 = sum([query[x] * dokumen3[x] for x in intersection2])  # A dot D
numerator7 = sum([query[x] * dokumen1[x] for x in intersection1])  # A dot B
numerator8 = sum([query[x] * dokumen2[x] for x in intersection2])  # A dot C
numerator9 = sum([query[x] * dokumen3[x] for x in intersection2])  # A dot D
numerator10 = sum([query[x] * dokumen1[x] for x in intersection1])  # A dot B
numerator11 = sum([query[x] * dokumen2[x] for x in intersection2])  # A dot C
numerator12 = sum([query[x] * dokumen3[x] for x in intersection2])  # A dot D
numerator13 = sum([query[x] * dokumen1[x] for x in intersection1])  # A dot B
numerator14 = sum([query[x] * dokumen2[x] for x in intersection2])  # A dot C
numerator15 = sum([query[x] * dokumen3[x] for x in intersection2])  # A dot D
numerator16 = sum([query[x] * dokumen1[x] for x in intersection1])  # A dot B
numerator17 = sum([query[x] * dokumen2[x] for x in intersection2])  # A dot C
numerator18 = sum([query[x] * dokumen3[x] for x in intersection2])  # A dot D
numerator19 = sum([query[x] * dokumen1[x] for x in intersection1])  # A dot B
numerator20 = sum([query[x] * dokumen2[x] for x in intersection2])  # A dot C
numerator21 = sum([query[x] * dokumen3[x] for x in intersection2])  # A dot D
numerator22 = sum([query[x] * dokumen1[x] for x in intersection1])  # A dot B
numerator23 = sum([query[x] * dokumen2[x] for x in intersection2])  # A dot C
numerator24 = sum([query[x] * dokumen3[x] for x in intersection2])  # A dot D
numerator25 = sum([query[x] * dokumen2[x] for x in intersection2])  # A dot C
numerator26 = sum([query[x] * dokumen3[x] for x in intersection2])  # A dot D
numerator27 = sum([query[x] * dokumen1[x] for x in intersection1])  # A dot B
numerator28 = sum([query[x] * dokumen2[x] for x in intersection2])  # A dot C
numerator29 = sum([query[x] * dokumen3[x] for x in intersection2])  # A dot D



# A dot B
# A = 0, 1, 3, 4
# B = 1, 0, 1, 2
# --------------- A dot B
# Z = 0, 0, 3, 8 -> 11

In [ ]:
sum_query = sum([query[x]**2 for x in query.keys()])
print("query:",sum_query)

sum_dok1 = sum([dokumen1[x]**2 for x in dokumen1.keys()])
print("dokumen1:",sum_dok1)

sum_dok2 = sum([dokumen2[x]**2 for x in dokumen2.keys()])
print("dokumen2:",sum_dok2)

sum_dok3 = sum([dokumen3[x]**2 for x in dokumen3.keys()])
print("dokumen3:",sum_dok3)

sum_dok4 = sum([dokumen4[x]**2 for x in dokumen4.keys()])
print("dokumen4:",sum_dok4)

sum_dok5 = sum([dokumen5[x]**2 for x in dokumen5.keys()])
print("dokumen5:",sum_dok5)

sum_dok6 = sum([dokumen6[x]**2 for x in dokumen6.keys()])
print("dokumen6:",sum_dok6)

sum_dok7 = sum([dokumen7[x]**2 for x in dokumen7.keys()])
print("dokumen7:",sum_dok7)

sum_dok8 = sum([dokumen8[x]**2 for x in dokumen8.keys()])
print("dokumen8:",sum_dok8)

sum_dok9 = sum([dokumen9[x]**2 for x in dokumen9.keys()])
print("dokumen9:",sum_dok9)

sum_dok10 = sum([dokumen10[x]**2 for x in dokumen10.keys()])
print("dokumen10:",sum_dok10)

sum_dok11 = sum([dokumen11[x]**2 for x in dokumen11.keys()])
print("dokumen11:",sum_dok11)

sum_dok12 = sum([dokumen12[x]**2 for x in dokumen12.keys()])
print("dokumen12:",sum_dok12)

sum_dok13 = sum([dokumen13[x]**2 for x in dokumen13.keys()])
print("dokumen13:",sum_dok13)

sum_dok14 = sum([dokumen14[x]**2 for x in dokumen14.keys()])
print("dokumen14:",sum_dok14)

sum_dok15 = sum([dokumen15[x]**2 for x in dokumen15.keys()])
print("dokumen15:",sum_dok15)

sum_dok16 = sum([dokumen16[x]**2 for x in dokumen16.keys()])
print("dokumen16:",sum_dok16)

sum_dok17 = sum([dokumen17[x]**2 for x in dokumen17.keys()])
print("dokumen17:",sum_dok17)

sum_dok18 = sum([dokumen18[x]**2 for x in dokumen18.keys()])
print("dokumen18:",sum_dok18)

sum_dok19 = sum([dokumen19[x]**2 for x in dokumen19.keys()])
print("dokumen19:",sum_dok19)

sum_dok20 = sum([dokumen20[x]**2 for x in dokumen20.keys()])
print("dokumen20:",sum_dok20)

sum_dok21 = sum([dokumen21[x]**2 for x in dokumen21.keys()])
print("dokumen21:",sum_dok21)

sum_dok22 = sum([dokumen22[x]**2 for x in dokumen22.keys()])
print("dokumen22:",sum_dok22)

sum_dok21 = sum([dokumen21[x]**2 for x in dokumen21.keys()])
print("dokumen21:",sum_dok21)

sum_dok22 = sum([dokumen22[x]**2 for x in dokumen22.keys()])
print("dokumen22:",sum_dok22)

sum_dok23 = sum([dokumen23[x]**2 for x in dokumen23.keys()])
print("dokumen23:",sum_dok23)

sum_dok24 = sum([dokumen24[x]**2 for x in dokumen24.keys()])
print("dokumen24:",sum_dok24)

sum_dok25 = sum([dokumen25[x]**2 for x in dokumen25.keys()])
print("dokumen25:",sum_dok25)

sum_dok26 = sum([dokumen26[x]**2 for x in dokumen26.keys()])
print("dokumen26:",sum_dok26)

sum_dok27 = sum([dokumen27[x]**2 for x in dokumen27.keys()])
print("dokumen21:",sum_dok27)

sum_dok28 = sum([dokumen28[x]**2 for x in dokumen28.keys()])
print("dokumen28:",sum_dok28)

sum_dok29 = sum([dokumen29[x]**2 for x in dokumen29.keys()])
print("dokumen29:",sum_dok29)

In [ ]:
denominator1 = math.sqrt(sum_query) * math.sqrt(sum_dok1) # ||A||.||B||
print("denominator1:",denominator1)

denominator2 = math.sqrt(sum_query) * math.sqrt(sum_dok2) # ||A||.||C||
print("denominator2:",denominator2)

denominator3 = math.sqrt(sum_query) * math.sqrt(sum_dok3) # ||A||.||D||
print("denominator2:",denominator2)

denominator4 = math.sqrt(sum_query) * math.sqrt(sum_dok4) # ||A||.||D||
print("denominator4:",denominator4)

denominator5 = math.sqrt(sum_query) * math.sqrt(sum_dok5) # ||A||.||D||
print("denominator5:",denominator5)

denominator6 = math.sqrt(sum_query) * math.sqrt(sum_dok6) # ||A||.||D||
print("denominator6:",denominator6)

denominator7 = math.sqrt(sum_query) * math.sqrt(sum_dok7) # ||A||.||D||
print("denominator7:",denominator7)

denominator8 = math.sqrt(sum_query) * math.sqrt(sum_dok8) # ||A||.||D||
print("denominator8:",denominator8)

denominator9 = math.sqrt(sum_query) * math.sqrt(sum_dok9) # ||A||.||D||
print("denominator9:",denominator9)

denominator10 = math.sqrt(sum_query) * math.sqrt(sum_dok10) # ||A||.||D||
print("denominator10:",denominator10)

denominator11 = math.sqrt(sum_query) * math.sqrt(sum_dok11) # ||A||.||D||
print("denominator11:",denominator11)

denominator12 = math.sqrt(sum_query) * math.sqrt(sum_dok12) # ||A||.||D||
print("denominator12:",denominator12)

denominator13 = math.sqrt(sum_query) * math.sqrt(sum_dok13) # ||A||.||D||
print("denominator13:",denominator13)

denominator14 = math.sqrt(sum_query) * math.sqrt(sum_dok14) # ||A||.||D||
print("denominator14:",denominator14)

denominator15 = math.sqrt(sum_query) * math.sqrt(sum_dok15) # ||A||.||D||
print("denominator15:",denominator15)

denominator16 = math.sqrt(sum_query) * math.sqrt(sum_dok16) # ||A||.||D||
print("denominator16:",denominator16)

denominator17 = math.sqrt(sum_query) * math.sqrt(sum_dok17) # ||A||.||D||
print("denominator17:",denominator17)

denominator18 = math.sqrt(sum_query) * math.sqrt(sum_dok18) # ||A||.||D||
print("denominator18:",denominator18)

denominator19 = math.sqrt(sum_query) * math.sqrt(sum_dok19) # ||A||.||D||
print("denominator19:",denominator19)

denominator20 = math.sqrt(sum_query) * math.sqrt(sum_dok20) # ||A||.||D||
print("denominator20:",denominator20)

denominator21 = math.sqrt(sum_query) * math.sqrt(sum_dok21) # ||A||.||D||
print("denominator21:",denominator21)

denominator22 = math.sqrt(sum_query) * math.sqrt(sum_dok22) # ||A||.||D||
print("denominator22:",denominator22)

denominator23 = math.sqrt(sum_query) * math.sqrt(sum_dok23) # ||A||.||D||
print("denominator23:",denominator23)

denominator24 = math.sqrt(sum_query) * math.sqrt(sum_dok24) # ||A||.||D||
print("denominator24:",denominator24)

denominator25 = math.sqrt(sum_query) * math.sqrt(sum_dok25) # ||A||.||D||
print("denominator25:",denominator25)

denominator26 = math.sqrt(sum_query) * math.sqrt(sum_dok26) # ||A||.||D||
print("denominator26:",denominator26)

denominator27 = math.sqrt(sum_query) * math.sqrt(sum_dok27) # ||A||.||D||
print("denominator27:",denominator27)

denominator28 = math.sqrt(sum_query) * math.sqrt(sum_dok28) # ||A||.||D||
print("denominator28:",denominator28)

denominator29 = math.sqrt(sum_query) * math.sqrt(sum_dok29) # ||A||.||D||
print("denominator29:",denominator29)

In [ ]:
# Cosine(A, B)

if not denominator1:
    cosine1 = 0.0
else:
    cosine1 = float(numerator1) / denominator1
    
print(cosine1)

# Cosine(A, C)

if not denominator2:
    cosine2 = 0.0
else:
    cosine2 = float(numerator2) / denominator2

print(cosine2)

# Cosine(A, D)

if not denominator3:
    cosine3 = 0.0
else:
    cosine3 = float(numerator3) / denominator3

print(cosine3)

if not denominator4:
    cosine4 = 0.0
else:
    cosine4 = float(numerator4) / denominator4

print(cosine4)

if not denominator5:
    cosine5 = 0.0
else:
    cosine5 = float(numerator5) / denominator5

print(cosine5)

if not denominator6:
    cosine6 = 0.0
else:
    cosine6 = float(numerator6) / denominator6

print(cosine6)

if not denominator7:
    cosine7 = 0.0
else:
    cosine7 = float(numerator7) / denominator7

print(cosine7)

if not denominator8:
    cosine8 = 0.0
else:
    cosine8 = float(numerator8) / denominator8

print(cosine8)

if not denominator9:
    cosine9 = 0.0
else:
    cosine9 = float(numerator9) / denominator9

print(cosine9)

if not denominator10:
    cosine10 = 0.0
else:
    cosine10 = float(numerator10) / denominator10

print(cosine10)

if not denominator11:
    cosine11 = 0.0
else:
    cosine11 = float(numerator11) / denominator11

print(cosine11)

if not denominator12:
    cosine12 = 0.0
else:
    cosine12 = float(numerator12) / denominator12

print(cosine12)

if not denominator13:
    cosine13 = 0.0
else:
    cosine13 = float(numerator13) / denominator13

print(cosine13)

if not denominator14:
    cosine14 = 0.0
else:
    cosine14 = float(numerator14) / denominator14

print(cosine14)

if not denominator15:
    cosine15 = 0.0
else:
    cosine15 = float(numerator15) / denominator15

print(cosine15)

if not denominator16:
    cosine16 = 0.0
else:
    cosine16 = float(numerator16) / denominator16

print(cosine16)

if not denominator17:
    cosine17 = 0.0
else:
    cosine17 = float(numerator17) / denominator17

print(cosine17)

if not denominator18:
    cosine18 = 0.0
else:
    cosine18 = float(numerator18) / denominator18

print(cosine18)

if not denominator19:
    cosine19 = 0.0
else:
    cosine19 = float(numerator19) / denominator19

print(cosine19)

if not denominator20:
    cosine20 = 0.0
else:
    cosine20 = float(numerator20) / denominator20

print(cosine20)

if not denominator21:
    cosine21 = 0.0
else:
    cosine21 = float(numerator21) / denominator21

print(cosine21)

if not denominator22:
    cosine22 = 0.0
else:
    cosine22 = float(numerator22) / denominator22

print(cosine22)

if not denominator23:
    cosine23 = 0.0
else:
    cosine23 = float(numerator23) / denominator23

print(cosine23)

if not denominator24:
    cosine24 = 0.0
else:
    cosine24 = float(numerator24) / denominator24

print(cosine24)

if not denominator25:
    cosine25 = 0.0
else:
    cosine25 = float(numerator25) / denominator25

print(cosine25)

if not denominator26:
    cosine26 = 0.0
else:
    cosine26 = float(numerator26) / denominator26

print(cosine26)

if not denominator27:
    cosine27 = 0.0
else:
    cosine27 = float(numerator27) / denominator27

print(cosine27)

if not denominator28:
    cosine28 = 0.0
else:
    cosine28 = float(numerator28) / denominator28

print(cosine28)

if not denominator29:
    cosine29 = 0.0
else:
    cosine29 = float(numerator29) / denominator29

print(cosine29)

In [ ]:
def bubbleSort(arr, dok):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] < arr[j+1] :
                dok[j], dok[j+1] = dok[j+1], dok[j]
                arr[j], arr[j+1] = arr[j+1], arr[j]
                
 
# Driver code to test above
arr = [cosine1, cosine2, cosine3,cosine4, cosine5, cosine6,cosine7, cosine8, cosine9, cosine10, cosine11, cosine12, cosine13, cosine14,cosine15, cosine16, cosine17,cosine18, cosine19, cosine20,cosine21, cosine22, cosine23,cosine24, cosine25, cosine26,cosine27, cosine28, cosine29]
dok = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
urut = {1:'Dokumen 1', 2:'Dokumen 2', 3:'Dokumen 3', 4:'Dokumen 4', 5:'Dokumen 5', 6:'Dokumen 6', 7:'Dokumen 7', 8:'Dokumen 8', 9:'Dokumen 9', 10:'Dokumen 10', 11:'Dokumen 11', 12:'Dokumen 12', 13:'Dokumen 13', 14:'Dokumen 14', 15:'Dokumen 15', 16:'Dokumen 16', 17:'Dokumen 17', 18:'Dokumen 18',19:'Dokumen 19', 20:'Dokumen 20', 21:'Dokumen 21', 22:'Dokumen 22', 23:'Dokumen 23', 24:'Dokumen 24', 25:'Dokumen 25', 26:'Dokumen 26', 27:'Dokumen 27', 28:'Dokumen 28', 29:'Dokumen 29'}
bubbleSort(arr, dok)

print ("Urutan ranking dokumen dari terbesar ke terkecil adalah:")

for d in range(29):
    print("Dokumen : {}".format(urut[dok[d]]))
    print("Nilai :{}".format(arr[d]))